<div align="center">
    <h1>DS-210: Programming for Data Science</h1>
    <h1>Lecture 30</h1>
</div>

# 1. Code formatting
# 2. Priority queues
# 3. Popular implementation: binary heap

# <font color="red">1. Code formatting</font>
# 2. Priority queues
# 3. Popular implementation: binary heap

## Don't give up on code formatting!

* Rust doesn't require any specific indentation
* Still a good idea to make your code readable

In [2]:
 fn h(z:i32)->i32{let mut t=0.max(z.min(1)-0.max(z-1));for y in 1..=2.min(z){t+=h(z-y)}t}

In [3]:
for i in 0..10 {
    println!("{}:{}",i,h(i));
};

0:0
1:1
2:1
3:2
4:3
5:5
6:8
7:13
8:21
9:34


## Tool for formatting Rust code: `rustfmt`

* If you have Rust installed, you should already have it.

* `rustfmt [filename]` replaces the file with nicely formatted version
   * use `rustfmt --backup [filename]` to save the original file

<div align="center">
    <b>[see demo with comparison via kdiff3]</b>
</div>

* `rustfmt --help`: see the command line parameters
* `rustfmt --print-config default`: default config that can be adjusted

# 1. Code formatting
# <font color="red">2. Priority queues</font>
# 3. Popular implementation: binary heap

## Priority queues

**Standard queue:**

* things returned in order in which they were inserted

**Priority queue:**
  * items have priorities
  * highest priority items returned first

## Rust standard library implementation: `BinaryHeap<T>`

<br>

* Priorities provided by the ordering of elements of `T` (via trait `Ord`)


<br>
<br>

* Method `push(T)`:<br>push element onto the heap

<br>
<br>


* Method `pop() -> Option<T>`:<br>remove the greatest and return it

In [9]:
use std::collections::BinaryHeap;

let mut pq = BinaryHeap::new();

pq.push(2);
pq.push(7);
pq.push(3);

println!("{:?}",pq.pop());
println!("{:?}",pq.pop());

pq.push(3);
pq.push(4);

println!("\n{:?}",pq.pop());
println!("{:?}",pq.pop());
println!("{:?}",pq.pop());
println!("{:?}",pq.pop());

Some(7)
Some(3)

Some(4)
Some(3)
Some(2)
None


## Getting the smallest element out first

`Reverse<T>`: wrapper that reverses the ordering of elements of a type

In [10]:
3 < 4

true

In [11]:
use std::cmp::Reverse;
Reverse(3) < Reverse(4)

false

In [12]:
5 < 3

false

In [13]:
Reverse(5) < Reverse(3)

true

In [14]:
let mut pq = BinaryHeap::new();

pq.push(Reverse(3));
pq.push(Reverse(1));
pq.push(Reverse(7));

println!("{:?}",pq.pop());
println!("{:?}",pq.pop());

pq.push(Reverse(0));

println!("\n{:?}",pq.pop());

Some(Reverse(1))
Some(Reverse(3))

Some(Reverse(0))


## Default lexicographic ordering on tuples and structs

Lexicographic ordering:
* Compare first elements
* If equal, compare second elements
* If equal, compare third elements...

### Tuples

In [15]:
(3,4) < (2,7)

false

In [16]:
(11,2,7) < (11,3,4)

true

### Struct (derive `Ord`)

In [17]:
#[derive(PartialEq,Eq,PartialOrd,Ord,Debug)]
struct Point {
    x: i32,
    y: i32,
}

In [18]:
let p = Point{x:3,y:4};
let q = Point{x:2,y:7};
println!("{}", p < q);
println!("{}", p > q);

false
true


## Another option: implement your own comparison

* More complicated, see below

* See the documentation for `Ord` or examples online

In [30]:
#[derive(PartialEq,Eq,Ord,Debug)]
struct Point2 {
    x: i32,
    y: i32,
}

// Let's assume we want to compare point by their distance to the origin
impl std::cmp::PartialOrd for Point2 {
    fn partial_cmp(&self, other: &Self) -> Option<std::cmp::Ordering> {
        let this = self.x * self.x + self.y + self.y;
        let that = other.x * other.x + other.y * other.y;
        return this.partial_cmp(&that);
    }
}


In [31]:
let p = Point2{x:3,y:1};
let q = Point2{x:2,y:100};
println!("{}", p < q);
println!("{}", p > q);

true
false


## How to implement a priority queue?
Assumptions:
 * At most $n$ elements
 * Comparison takes $O(1)$ time

### Straighforward

Representation: a vector of elements

Push:
* add to the end of the vector
* Time complexity: $O(1)$ (amortized) time

Pop:
* go over all elements, select the greatest
* Time complexity: $O(n)$

# 1. Code formatting
# 2. Priority queues
# <font color="red">3. Popular implementation: binary heap</font>

## Binary heaps

* Data organized into a binary tree
* Every internal node not smaller than its children

**Basic property:**
The root has the current maximum (minimum), i.e., the answer to next `pop`

<div align="center">
    <img src="order.png" alt="[picture of basic binary heap with heap ordering]" width="70%">
</div>

## Binary heaps

**Efficient storage:**
* Tree levels filled from left to right
* Can be mapped to a vector

<div align="center">
    <img src="layers.png" alt="[picture of basic binary heap with heap ordering]" width="100%">
</div>

* Easy to move to the parent or children using vector indices

<div align="center">
    <img src="indices.png" alt="[picture of basic binary heap with heap ordering]" width="50%">
</div>

## How are operations implemented?

### Push

* add at the end the array
* fix the ordering by pushing the element up

### Pop

* remove and return the root
* replace with the last element
* fix the ordering, pushing the element down

### Complexity of push and pop

* Proportional to the number of levels

* So $O(\log n)$


In [2]:
#[derive(Debug)]
struct BinaryHeap {
    heap: Vec<i32>,
    heap_size: usize,
}

impl BinaryHeap {
    fn new() -> BinaryHeap {
        let heap: Vec<i32> = vec![];
        let heap_size = 0;
        BinaryHeap { heap, heap_size }
    }

    fn left(i: usize) -> usize {
        2 * i + 1
    }

    fn right(i: usize) -> usize {
        2 * i + 2
    }

    fn parent(i: usize) -> usize {
        (i - 1) / 2
    }
}

In [3]:
impl BinaryHeap {
        fn heapify(&mut self, loc: usize) {
        let l = Self::left(loc);
        let r: usize = Self::right(loc);
        let mut largest = loc;
        if l < self.heap_size && self.heap[l] > self.heap[largest] {
            largest = l;
        }
        if r < self.heap_size && self.heap[r] > self.heap[largest] {
            largest = r;
        }
        if largest != loc {
            let tmp = self.heap[loc];
            self.heap[loc] = self.heap[largest];
            self.heap[largest] = tmp;
            self.heapify(largest);
        }
    }
}

In [4]:
impl BinaryHeap {
        fn insert_val(&mut self, val: i32) {
        self.heap_size += 1;
        self.heap.push(val);
        let mut i = self.heap_size - 1;
        while i != 0 && self.heap[Self::parent(i)] < self.heap[i] {
            let tmp = self.heap[Self::parent(i)];
            self.heap[Self::parent(i)] = self.heap[i];
            self.heap[i] = tmp;
            i = Self::parent(i);
        }
    }
    
    fn extract_max(&mut self) -> i32 {
        if self.heap_size == 0 {
            return i32::MIN;
        }
        if self.heap_size == 1 {
            self.heap_size -= 1;
            return self.heap[0];
        }
        let root = self.heap[0];
        self.heap[0] = self.heap[self.heap_size - 1];
        self.heap_size -= 1;
        self.heapify(0);
        return root;
    }
}

In [6]:
:dep rand="0.8.5"
use rand::Rng;

let mut h = BinaryHeap::new();
for _i in 0..10 {
    let x = rand::thread_rng().gen_range(-1000..1000) as i32;
    h.insert_val(x);
}
println!("{:?}", h);
let size = h.heap_size;
for _j in 0..size {
    let z = h.extract_max();
    print!("{} ", z);
}
println!("");
println!("{:?}", h);


BinaryHeap { heap: [771, 665, 539, 196, 372, 109, -775, -834, -566, 229], heap_size: 10 }
771 665 539 372 229 196 109 -566 -775 -834 
BinaryHeap { heap: [-834, -834, -834, -775, -834, -566, -775, -834, -566, 229], heap_size: 0 }
